In [1]:
import gym
import math
import random
import numpy as np
import pandas as pd

from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import time

import collections
import copy

#env = gym.make('CartPole-v0').unwrapped

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# if gpu is to be used
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=1)

In [2]:
data_path = '/home/ubuntu/data/dataset/R3009_U5_V100/'
UIT = pd.read_csv(data_path + 'UIT.csv')
UIT

,u,i,day,time,viewtime,video_type,video_format,city,city_isp,client_ip,conn_type,device_type
0,365,3391,0,0,148,1030,101001,0,0,11807,1,2
1,203,5779,0,0,7,1030,10203,0,0,15068,1,2
2,208,4675,0,0,92,1035,10203,0,0,5375,1,2
3,159,332,0,0,56,1030,10202,0,0,5992,1,2
4,50,674,0,0,439,1030,10203,0,0,3468,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
300978,483,6831,29,2591880,34,1030,10203,0,0,10010,1,2
300979,158,8448,29,2591880,34,1030,10203,0,0,23340,1,2
300980,483,6463,29,2591940,35,1030,10203,0,0,10010,1,2
300981,158,4715,29,2591940,34,1030,10203,0,0,23340,1,2


In [3]:
trainUIT = UIT[UIT['day']<18]
contentNum = len(UIT.i.drop_duplicates())
userNum = len(UIT.u.drop_duplicates())
contentNum,userNum,trainUIT

(10000,
 500,
           u     i  day     time  viewtime  video_type  video_format  city  \
 0       365  3391    0        0       148        1030        101001     0   
 1       203  5779    0        0         7        1030         10203     0   
 2       208  4675    0        0        92        1035         10203     0   
 3       159   332    0        0        56        1030         10202     0   
 4        50   674    0        0       439        1030         10203     0   
 ...     ...   ...  ...      ...       ...         ...           ...   ...   
 198170  264  7442   17  1555140        90        1035         10203     0   
 198171   19  9362   17  1555140       424        1035         10203     0   
 198172   82  9223   17  1555140        94        1037         10203     0   
 198173   35  4164   17  1555140        22        1030         10203     0   
 198174  239  5062   17  1555140        89        1035         10203     0   
 
         city_isp  client_ip  conn_type  device_

In [4]:
class ENV(object):
    def __init__(self,userNum,contentNum):
        self.userNum = userNum
        self.contentNum =contentNum

        self.r = np.zeros(shape=(userNum,contentNum),dtype=int)
        self.p = np.full(shape=contentNum,fill_value = 1/userNum)
        self.e = np.zeros(shape=contentNum)
        self.S = np.ones(shape=contentNum,dtype=int)
        self.l_edge = 0.1
        self.l_cp = 1

        self.B = np.full(shape=userNum,fill_value=10,dtype=int)

        self.pipe = collections.OrderedDict()


    #有序字典实现LRU
    def updateEgdeCache(self,action,t):
        for i in np.argwhere(action==1).squeeze(-1):
            if i in self.pipe.keys():
                self.pipe.pop(i)
            elif len(self.pipe) >= 500:
                self.e[self.pipe.popitem(last=False)[0]] = 0
            self.pipe[i] = t
            self.e[i] = 1

    
    def updateEnv(self,u,action,t):
        
        p_tmp = ((self.r[u] | action)-self.r[u])*(1/self.userNum) + self.p
        self.p = np.where(p_tmp<1-1/self.userNum,p_tmp,1-1/self.userNum)

        self.r[u] = self.r[u] | action

        self.updateEgdeCache(action,t)

    def getStatus(self):
        return (torch.from_numpy(self.r),
                torch.from_numpy(self.p) , 
                torch.from_numpy(self.e),
                torch.from_numpy(self.S),
                self.l_edge,
                self.l_cp)

    #def reset(self):
    #    self.r = np.zeros(shape=(self.userNum,self.contentNum),dtype=int)
    #    self.p = np.full(shape=self.contentNum,fill_value = 1/self.userNum)
    #    self.e = np.zeros(shape=self.contentNum)
    #    self.S = np.ones(shape=self.contentNum,dtype=int)
    #    self.l_edge = 0.1
    #    self.l_cp = 1
    #    self.B = np.full(shape=self.userNum,fill_value=15,dtype=int)
    #    self.pipe = collections.OrderedDict()


In [5]:
class UE_random(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = torch.zeros(size=(env.contentNum,),dtype=int)

        self.Bu = int(env.B[self.u])
        self.contentNum = env.contentNum
        self.userNum = env.userNum

        self.r , self.p , self.e, self.S,self.l_edge, self.l_cp = env.getStatus()

        self.action = torch.zeros(size=(env.contentNum,),dtype=int)
        self.lastAction = self.action

        self.reward = 0
        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']

    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1

    
    def getReward(self,lastru,lastp,ru,p,i,action,S,Bu,l_edge,l_cp,e):

        self.Rh = - self.ALPHAh * (torch.log(ru * p + (1-ru) * (1-p)) - torch.log(lastru * lastp + (1-lastru) * (1-lastp))).sum()

        self.Ro =   self.BETAo * action[i] * (S[i] / Bu + ( e[i] * l_edge + ( 1-e[i] ) * l_cp ) / S[i])

        self.Rl =   self.BETAl * ( ( 1 - action[i] )  * ( l_cp - ( e[i] * l_edge + ( 1 - e[i] ) * l_cp ) ) ) / S[i]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        return  self.Rh+self.Ro+self.Rl

    def selectAction(self,env,uit):


        self.lastAction = self.action
        self.lastp = self.p
        self.lastr = self.r

        self.updateViewContent(uit[1])
        self.r , self.p , self.e, self.S, self.l_edge, self.l_cp = env.getStatus()
        
        self.reward = self.getReward(self.lastr[self.u],self.lastp,self.r[self.u],self.p,self.W[-1],self.lastAction,self.S,self.Bu,self.l_edge,self.l_cp,self.e)
        
        self.action = torch.zeros(size=(env.contentNum,),dtype=int)
        self.action[self.W[-1]] = 1

        actionIndex = list(torch.randint(0,self.contentNum,(self.Bu,)))
        
        if self.W[-1] not in actionIndex:
            actionIndex.pop()
        for index in actionIndex:
            self.action[index] = 1

        env.updateEnv(self.u,self.action.numpy(),uit[2])

        return self.action

In [6]:
num_episodes = 1
TARGET_UPDATE = 1
bestReward =  float("-inf")

env = ENV(userNum,contentNum)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

rewardPara = {"alpha":1,"betao":0.5,"betal":0.5}

sumHitrate = np.zeros(UIT.shape[0]// 10000 +2)
UEHitrate = np.zeros(UIT.shape[0]// 10000 +2)
edgeHitrate = np.zeros(UIT.shape[0]// 10000 +2)
privacyReduction = np.zeros(UIT.shape[0]// 10000 +2)


for i_episode in range(num_episodes):
    # Initialize the environment and state

    for index,trace in UIT.iterrows():
        uit = trace.to_numpy()
        if uit[0] not in UEs:
            UEs[uit[0]] = UE_random(uit[0],env,rewardPara)

        ue = UEs[uit[0]]
        
        actionIndex = np.argwhere(ue.lastAction)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit)

        sumReward[0] += float(ue.Rh)
        sumReward[1] += float(ue.Rl)
        sumReward[2] += float(ue.Ro)

        if index % 10000 == 0 :
            psi = 0
            p = torch.from_numpy(env.p)
            for u in UEs:
                psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
            print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
            print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
            print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
            print()
            sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
            UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
            edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
            privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    print()
    print("----------------------------------------------------------------")
    print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
    print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
    print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
    print("----------------------------------------------------------------")
    print()

    sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
    UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
    edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
    privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Sun Sep 19 13:50:48 2021 Episode: 0   Index: 0   Loss: 0.0 --
Reward: [0. 0. 0.] total reward: 0.0
UEHitrate: 0.0  edgeHitrate 0.0 sumHitrate 0.0  privacy: 2.94295

--Time: Sun Sep 19 13:51:01 2021 Episode: 0   Index: 10000   Loss: 0.0 --
Reward: [-3.55515e+00  4.43200e-02  1.63000e-03] total reward: -3.5092
UEHitrate: 0.0021  edgeHitrate 0.09869 sumHitrate 0.10079  privacy: 3.56311

--Time: Sun Sep 19 13:51:14 2021 Episode: 0   Index: 20000   Loss: 0.0 --
Reward: [-2.99492e+00  4.90000e-02  2.05000e-03] total reward: -2.94387
UEHitrate: 0.0024  edgeHitrate 0.10889 sumHitrate 0.11129  privacy: 3.31123

--Time: Sun Sep 19 13:51:27 2021 Episode: 0   Index: 30000   Loss: 0.0 --
Reward: [-2.5363e+00  4.5720e-02  2.2500e-03] total reward: -2.48833
UEHitrate: 0.00273  edgeHitrate 0.1019 sumHitrate 0.10463  privacy: 3.08386

--Time: Sun Sep 19 13:51:40 2021 Episode: 0   Index: 40000   Loss: 0.0 --
Reward: [-2.28306  0.04521  0.00231] total reward: -2.23553
UEHitrate: 0.00302  edgeHitr

In [7]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.     , 0.10079, 0.11129, 0.10463, 0.10372, 0.10394, 0.10135,
        0.09814, 0.09689, 0.09655, 0.09581, 0.09576, 0.09573, 0.09478,
        0.09513, 0.09521, 0.09436, 0.09462, 0.0948 , 0.0944 , 0.0946 ,
        0.09498, 0.09458, 0.09457, 0.09424, 0.09484, 0.09455, 0.09412,
        0.09448, 0.0949 , 0.09497, 0.     ]),
 array([0.     , 0.0021 , 0.0024 , 0.00273, 0.00302, 0.0033 , 0.00315,
        0.00314, 0.0031 , 0.00324, 0.00327, 0.0033 , 0.00341, 0.0036 ,
        0.00364, 0.00371, 0.00371, 0.00375, 0.00373, 0.00382, 0.0038 ,
        0.00387, 0.00391, 0.00396, 0.00398, 0.00404, 0.00409, 0.00415,
        0.00417, 0.0042 , 0.00419, 0.     ]),
 array([0.     , 0.09869, 0.10889, 0.1019 , 0.1007 , 0.10064, 0.0982 ,
        0.095  , 0.09379, 0.09331, 0.09254, 0.09246, 0.09232, 0.09118,
        0.09149, 0.09151, 0.09066, 0.09087, 0.09107, 0.09059, 0.0908 ,
        0.09111, 0.09066, 0.09062, 0.09025, 0.0908 , 0.09046, 0.08997,
        0.0903 , 0.0907 , 0.09077, 0.     ]))

In [8]:
privacyReduction

array([2.94295, 3.56311, 3.31123, 3.08386, 2.91028, 2.74719, 2.62383,
       2.52184, 2.42206, 2.32575, 2.2257 , 2.14044, 2.07087, 2.00069,
       1.9371 , 1.87852, 1.81909, 1.764  , 1.7077 , 1.6565 , 1.60888,
       1.56053, 1.51605, 1.47154, 1.43023, 1.39058, 1.35325, 1.31744,
       1.28244, 1.2482 , 1.21305, 0.     ])

In [8]:
class UE_None(object):
    def __init__(self,u,env,rewardPara):
        self.u = u

        self.W = []
        self.v = torch.zeros(size=(env.contentNum,),dtype=int)

        self.Bu = int(env.B[self.u])
        self.contentNum = env.contentNum
        self.userNum = env.userNum

        self.r , self.p , self.e, self.S,self.l_edge, self.l_cp = env.getStatus()

        self.action = torch.zeros(size=(env.contentNum,),dtype=int)
        self.lastAction = self.action

        self.reward = 0
        self.ALPHAh = rewardPara['alpha']
        self.BETAo =  rewardPara['betao']
        self.BETAl =  rewardPara['betal']

    def updateViewContent(self,i):
        self.W.append(i)
        self.v[i] = 1

    
    def getReward(self,lastru,lastp,ru,p,i,action,S,Bu,l_edge,l_cp,e):

        self.Rh = - self.ALPHAh * (torch.log(lastru * lastp + (1-lastru) * (1-lastp)).sum() - torch.log(ru * p + (1-ru) * (1-p)).sum())

        self.Ro =   self.BETAo * action[i] * (S[i] / Bu + ( e[i] * l_edge + ( 1-e[i] ) * l_cp ) / S[i])

        self.Rl =   self.BETAl * ( ( 1 - action[i] )  * ( l_cp - ( e[i] * l_edge + ( 1 - e[i] ) * l_cp ) ) ) / S[i]

        #self.Rh[i] = self.Rh[i] + self.Ro + self.Rl

        return  self.Rh+self.Ro+self.Rl

    def selectAction(self,env,uit):

        self.lastAction = self.action
        self.lastp = self.p
        self.lastr = self.r

        self.updateViewContent(uit[1])
        self.r , self.p , self.e, self.S, self.l_edge, self.l_cp = env.getStatus()
        
        self.reward = self.getReward(self.lastr[self.u],self.lastp,self.r[self.u],self.p,self.W[-1],self.lastAction,self.S,self.Bu,self.l_edge,self.l_cp,self.e)
        
        self.action = torch.zeros(size=(env.contentNum,),dtype=int)
        self.action[self.W[-1]] = 1

        env.updateEnv(self.u,self.action.numpy(),uit[2])

        return self.action

In [16]:
num_episodes = 1
TARGET_UPDATE = 1
bestReward =  float("-inf")

env = ENV(userNum,contentNum)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

rewardPara = {"alpha":1,"betao":0.5,"betal":0.5}


for i_episode in range(num_episodes):
    # Initialize the environment and state

    for index,trace in trainUIT.iterrows():
        uit = trace.to_numpy()
        if uit[0] not in UEs:
            UEs[uit[0]] = UE_None(uit[0],env,rewardPara)

        ue = UEs[uit[0]]
        
        actionIndex = np.argwhere(ue.lastAction)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit)

        sumReward[0] += float(ue.Rh)
        sumReward[1] += float(ue.Rl)
        sumReward[2] += float(ue.Ro)

        if index % 10000 == 0 :
            psi = 0
            p = torch.from_numpy(env.p)
            for u in UEs:
                psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
            print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
            print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
            print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
            print()
            sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
            UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
            edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)

    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    print()
    print("----------------------------------------------------------------")
    print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
    print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
    print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
    print("----------------------------------------------------------------")
    print()

    

    

--Time: Wed Sep 15 22:04:35 2021 Episode: 0   Index: 0   Loss: 0.0 --
Reward: [0. 0. 0.] total reward: 0.0
UEHitrate: 0.0  edgeHitrate 0.0 sumHitrate 0.0  privacy: 1.0

--Time: Wed Sep 15 22:04:45 2021 Episode: 0   Index: 10000   Loss: 0.0 --
Reward: [0.10938 0.1794  0.00074] total reward: 0.28952
UEHitrate: 0.0018  edgeHitrate 0.39936 sumHitrate 0.40116  privacy: 1.0

--Time: Wed Sep 15 22:04:55 2021 Episode: 0   Index: 20000   Loss: 0.0 --
Reward: [0.03867 0.18685 0.00087] total reward: 0.22639
UEHitrate: 0.00195  edgeHitrate 0.41593 sumHitrate 0.41788  privacy: 1.0

--Time: Wed Sep 15 22:05:05 2021 Episode: 0   Index: 30000   Loss: 0.0 --
Reward: [0.01672 0.19601 0.00083] total reward: 0.21356
UEHitrate: 0.0024  edgeHitrate 0.43615 sumHitrate 0.43855  privacy: 1.0

--Time: Wed Sep 15 22:05:15 2021 Episode: 0   Index: 40000   Loss: 0.0 --
Reward: [0.01123 0.19757 0.00101] total reward: 0.20981
UEHitrate: 0.00247  edgeHitrate 0.43974 sumHitrate 0.44221  privacy: 1.0

--Time: Wed Sep 1

In [25]:
num_episodes = 1
TARGET_UPDATE = 1
bestReward =  float("-inf")

env = ENV(userNum,contentNum)
UEs = {}
sumReward = np.zeros(3)
loss = 0
UEHit = np.zeros(userNum)
edgeHit = 0

rewardPara = {"alpha":1,"betao":0.5,"betal":0.5}

sumHitrate = np.zeros(10)
UEHitrate = np.zeros(10)
edgeHitrate = np.zeros(10)
privacyReduction = np.zeros(10)


for i_episode in range(num_episodes):
    # Initialize the environment and state

    for index,trace in UIT.iterrows():
        uit = trace.to_numpy()
        if uit[0] not in UEs:
            UEs[uit[0]] = UE_None(uit[0],env,rewardPara)

        ue = UEs[uit[0]]
        
        actionIndex = np.argwhere(ue.lastAction)
        if uit[1] in actionIndex:
            UEHit[uit[0]] += 1
        elif uit[1] in env.pipe.keys():
            edgeHit += 1

        ue.selectAction(env,uit)

        sumReward[0] += float(ue.Rh)
        sumReward[1] += float(ue.Rl)
        sumReward[2] += float(ue.Ro)

        if index % 10000 == 0 :
            psi = 0
            p = torch.from_numpy(env.p)
            for u in UEs:
                psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
            print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
            print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
            print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
            print()
            sumHitrate[int(index // 10000)]   = round((edgeHit+UEHit.sum())/(index+1),5)
            UEHitrate [int(index // 10000)]   = round(UEHit.sum()/(index+1),5)
            edgeHitrate [int(index // 10000)] = round(edgeHit/(index+1),5)
            privacyReduction [int(index // 10000)] = round(float(psi)/len(UEs),5)
    psi = 0
    p = torch.from_numpy(env.p)
    for u in UEs:
        psi += torch.log(UEs[u].r[u] * p + (1-UEs[u].r[u]) * (1-p)).sum() / torch.log(UEs[u].v * p + (1-UEs[u].v) * (1-p)).sum()
    print()
    print("----------------------------------------------------------------")
    print("--Time:",time.asctime( time.localtime(time.time())),"Episode:",i_episode,"  Index:",index,"  Loss:",round(loss/(index+1),5),"--")
    print("Reward:",np.around(sumReward/(index+1),5),"total reward:",round(sumReward.sum()/(index+1),5))
    print("UEHitrate:",round(UEHit.sum()/(index+1),5)," edgeHitrate",round(edgeHit/(index+1),5),"sumHitrate",round((edgeHit+UEHit.sum())/(index+1),5)," privacy:",round(float(psi)/len(UEs),5))
    print("----------------------------------------------------------------")
    print()

    sumHitrate [int(round(index / 10000,0))]  = round((edgeHit+UEHit.sum())/(index+1),5)
    UEHitrate  [int(round(index / 10000,0))]  = round(UEHit.sum()/(index+1),5)
    edgeHitrate[int(round(index / 10000,0))]  = round(edgeHit/(index+1),5)
    privacyReduction [int(round(index / 10000,0))] = round(float(psi)/len(UEs),5)

--Time: Wed Sep 15 22:59:36 2021 Episode: 0   Index: 0   Loss: 0.0 --
Reward: [0. 0. 0.] total reward: 0.0
UEHitrate: 0.0  edgeHitrate 0.0 sumHitrate 0.0  privacy: 1.0

--Time: Wed Sep 15 22:59:47 2021 Episode: 0   Index: 10000   Loss: 0.0 --
Reward: [0.10938 0.1794  0.00074] total reward: 0.28952
UEHitrate: 0.0018  edgeHitrate 0.39936 sumHitrate 0.40116  privacy: 1.0

--Time: Wed Sep 15 22:59:58 2021 Episode: 0   Index: 20000   Loss: 0.0 --
Reward: [0.03867 0.18685 0.00087] total reward: 0.22639
UEHitrate: 0.00195  edgeHitrate 0.41593 sumHitrate 0.41788  privacy: 1.0

--Time: Wed Sep 15 23:00:09 2021 Episode: 0   Index: 30000   Loss: 0.0 --
Reward: [0.01672 0.19601 0.00083] total reward: 0.21356
UEHitrate: 0.0024  edgeHitrate 0.43615 sumHitrate 0.43855  privacy: 1.0

--Time: Wed Sep 15 23:00:20 2021 Episode: 0   Index: 40000   Loss: 0.0 --
Reward: [0.01123 0.19757 0.00101] total reward: 0.20981
UEHitrate: 0.00247  edgeHitrate 0.43974 sumHitrate 0.44221  privacy: 1.0

--Time: Wed Sep 1

In [26]:
sumHitrate, UEHitrate, edgeHitrate

(array([0.     , 0.40116, 0.41788, 0.43855, 0.44221, 0.44403, 0.44093,
        0.44305, 0.44431, 0.45009]),
 array([0.     , 0.0018 , 0.00195, 0.0024 , 0.00247, 0.00246, 0.00258,
        0.0028 , 0.00297, 0.00298]),
 array([0.     , 0.39936, 0.41593, 0.43615, 0.43974, 0.44157, 0.43834,
        0.44025, 0.44133, 0.44711]))

In [27]:
privacyReduction

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])